In [1]:
import numpy as np
import pandas as pd

In [2]:
file = 'records.txt'

In [3]:
data = pd.read_csv(file,header=None,sep='] ',names=['date','text'])
data.head()

/anaconda3/envs/jb/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,date,text
0,[1518-05-29 00:00,Guard #1151 begins shift
1,[1518-05-12 00:02,Guard #439 begins shift
2,[1518-04-11 00:33,falls asleep
3,[1518-06-23 00:52,wakes up
4,[1518-10-25 00:48,falls asleep


In [4]:
data['date'] = data['date'].apply(lambda x: x.replace('[','').replace('1518','2018'))
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by='date')
data.head()

,date,text
246,2018-03-18 00:01:00,Guard #89 begins shift
431,2018-03-18 00:35:00,falls asleep
418,2018-03-18 00:59:00,wakes up
131,2018-03-19 00:02:00,Guard #787 begins shift
62,2018-03-19 00:24:00,falls asleep


In [6]:
results = pd.DataFrame(columns=['start','end'])
results

,start,end


In [8]:
for i in range(len(data.index)):
    if '#' in data['text'].iloc[i]:
        new_guard = int(data['text'].iloc[i].replace('Guard #','').replace(' begins shift',''))
    elif 'falls asleep' in data['text'].iloc[i]:
        results = results.append(pd.DataFrame({'start':data['date'].iloc[i].minute,'end':data['date'].iloc[i+1].minute-1},index=[new_guard]) )

results['diff'] = results['end'] - results['start']

In [9]:
results.head()

,start,end,diff
89,35,58,23
787,24,57,33
499,8,53,45
2089,32,43,11
2089,54,57,3


In [24]:
most_sleepy = (results.groupby(results.index).sum().sort_values('diff',ascending=False)).index[0]
new_results = results.loc[most_sleepy]
new_results.head()

,start,end,diff
499,8,53,45
499,12,44,32
499,8,27,19
499,10,36,26
499,43,55,12


In [25]:
minutes = []

for i in range(len(new_results)):
    for minute in range(60):
        if minute >= new_results.iloc[i]['start'] and minute <= new_results.iloc[i]['end']:
            minutes.append(minute)

minutes = pd.Series(minutes).value_counts().sort_values(ascending=False)

print('part #1 answer:',minutes.index[0] * most_sleepy)
print(minutes)

part #1 answer: 21956
44    13
36    12
43    12
45    12
40    12
41    12
37    11
22    11
42    11
39    11
23    11
35    11
20    11
25    10
53    10
19    10
21    10
24    10
34    10
27    10
26    10
33    10
38    10
32     9
29     9
18     9
17     9
28     9
30     9
31     9
46     9
50     9
51     9
52     9
47     8
48     8
49     8
16     8
15     8
13     7
54     7
12     7
10     7
14     7
55     6
11     6
9      6
8      6
56     5
6      3
5      3
57     3
7      3
4      2
2      1
1      1
3      1
0      1
dtype: int64


In [14]:
#part 2:

most_minute = pd.Series()

for i in range(len(results)):
    for minute in range(60):
        if minute >= results.iloc[i]['start'] and minute <= results.iloc[i]['end']:
            most_minute = most_minute.append(pd.Series([minute],index=[results.index[i]] ) )

top = most_minute.groupby(most_minute.index).value_counts().sort_values(ascending=False)

print('part #2 answer:',top.index[0][0] * top.index[0][1])

part #2 answer: 134511
